In [ ]:
# Sentiment Analysis do DataFrame "ws_agencia_sebrae_noticias.xlsx" com o modelo BERTimabu (BASE)

# Importar as bibliotecas necessárias
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import warnings

# Caminho do arquivo no computador
caminho_arquivo = r'C:\Users\Thairone Almeida\Google Drive\Thairone\Hashtag Treinamentos\Python Impressionador\Script\MBA_USP_ESALQ\Dados até 30 06 2024\ws_agencia_sebrae_noticias.xlsx'

# Caminhos dos modelos BERTimbau
caminho_bertimbau_base = r'C:\Users\Thairone Almeida\Google Drive\Thairone\Hashtag Treinamentos\Python Impressionador\Script\MBA_USP_ESALQ\BERTimbau_base_GoEmotions_portuguese'
caminho_bertimbau_large = r'C:\Users\Thairone Almeida\Google Drive\Thairone\Hashtag Treinamentos\Python Impressionador\Script\MBA_USP_ESALQ\BERTimbau_large_GoEmotions_portuguese'

# Carregar os dados do arquivo Excel
df = pd.read_excel(caminho_arquivo)

# Mapeamento de BERTimbau Base
mapa_sentimentos_base = {
    0: "admiração", 1: "diversão", 2: "raiva", 3: "aborrecimento", 4: "aprovação", 5: "zelo", 6: "confusão", 7: "curiosidade", 8: "desejo", 9: "decepção", 10: "desaprovação", 11: "nojo", 12: "constrangimento", 13: "entusiasmo", 14: "medo", 15: "gratidão", 16: "luto", 17: "alegria", 18: "amor", 19: "nervosismo", 20: "otimismo", 21: "orgulho", 22: "percepção", 23: "alívio", 24: "remorso", 25: "tristeza", 26: "surpresa", 27: "neutro"
}

# Mapeamento de BERTimbau Large
mapa_sentimentos_large = {
    0: "admiração", 1: "diversão", 2: "raiva", 3: "aborrecimento", 4: "aprovação", 5: "zelo", 6: "confusão", 7: "curiosidade", 8: "desejo", 9: "decepção", 10: "desaprovação", 11: "nojo", 12: "constrangimento", 13: "entusiasmo", 14: "medo", 15: "gratidão", 16: "luto", 17: "alegria", 18: "amor", 19: "nervosismo", 20: "otimismo", 21: "orgulho", 22: "realização", 23: "alívio", 24: "remorso", 25: "tristeza", 26: "surpresa", 27: "neutro"
}

# Escolher o modelo que deseja usar (base ou large)
usar_modelo_large = False  # Alterar para True se quiser usar o modelo large

# Carregar o modelo e o tokenizer BERTimbau
if usar_modelo_large:
    tokenizer = BertTokenizer.from_pretrained(caminho_bertimbau_large)
    model = BertForSequenceClassification.from_pretrained(caminho_bertimbau_large)
    mapa_sentimentos = mapa_sentimentos_large
else:
    tokenizer = BertTokenizer.from_pretrained(caminho_bertimbau_base)
    model = BertForSequenceClassification.from_pretrained(caminho_bertimbau_base)
    mapa_sentimentos = mapa_sentimentos_base

# Definir função para remover stopwords personalizadas
def remover_stopwords(texto, stopwords_personalizadas):
    # Garantir que o texto é uma string e tratar valores nulos
    if pd.isnull(texto):
        return ""
    palavras = str(texto).split()
    palavras_filtradas = [palavra for palavra in palavras if palavra.lower() not in stopwords_personalizadas]
    return ' '.join(palavras_filtradas)

# Lista de stopwords personalizadas
stopwords_personalizadas = [
    '0800', 'a', 'as', 'à', 'às', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'acerca', 'adeus', 'agora', 'ainda', 'algo', 'algumas', 'alguns', 'ali', 'além', 'alô', 'ambos', 'ampla', 'amplas', 'amplo', 'amplos', 'ano', 'ante', 'antes', 'ao', 'aos', 'apenas', 'apoio', 'após', 'aqui', 'assim', 'atrás', 'através', 'até', 'aí', 'baixo', 'bem', 'boa', 'boas', 'bom', 'bons', 'breve', 'cada', 'catorze', 'cedo', 'cento', 'certamente', 'certeza', 'cinco', 'cima', 'coisa', 'coisas', 'com', 'como', 'conselho', 'contudo', 'contra', 'cá', 'da', 'das', 'dado', 'dados', 'dela', 'delas', 'dele', 'deles', 'demais', 'debaixo', 'de', 'dentro', 'depois', 'desde', 'dessa', 'dessas', 'desse', 'desses', 'desta', 'destas', 'deste', 'destes', 'devendo', 'deve', 'devem', 'deverá', 'dez', 'dezoito', 'dia', 'diante', 'disse', 'disso', 'disto', 'dito', 'diz', 'dizem', 'dizer', 'do', 'dos', 'dois', 'doze', 'duas', 'dúvida', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'embora', 'em', 'empresa', 'empresas', 'enquanto', 'entre', 'então', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'está', 'estamos', 'estão', 'esta', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'evento', 'exemplo', 'falta', 'favor', 'faz', 'fazem', 'fazemos', 'fazendo', 'fazer', 'fazeres', 'faço', 'feita', 'feitas', 'feito', 'feitos', 'fim', 'final', 'fomos', 'fôramos', 'fora', 'for', 'foram', 'forma', 'formos', 'fosse', 'fossem', 'fôssemos', 'foi', 'fui', 'geral', 'grande', 'grandes', 'grátis', 'haja', 'hajam', 'hajamos', 'hão', 'haver', 'havemos', 'havia', 'hei', 'hoje', 'hora', 'horas', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvéssemos', 'há', 'https', 'isso', 'isto', 'já', 'la', 'lado', 'lhe', 'lhes', 'ligado', 'lo', 'local', 'logo', 'longe', 'lugar', 'lá', 'maior', 'maioria', 'maiorias', 'mal', 'mais', 'mas', 'me', 'meio', 'menor', 'menos', 'meses', 'mesma', 'mesmas', 'mesmo', 'mesmos', 'meu', 'meus', 'minha', 'minhas', 'mil', 'milhões', 'milhares', 'muito', 'muitas', 'muitos', 'na', 'nas', 'nada', 'não', 'naquela', 'naquelas', 'naquele', 'naqueles', 'nas', 'negócio', 'negócios', 'nem', 'nesta', 'nestas', 'nessa', 'nessas', 'nesse', 'nesses', 'ninguém', 'nível', 'no', 'nos', 'noite', 'nome', 'nossa', 'nossas', 'nosso', 'nossos', 'nova', 'novas', 'nove', 'novo', 'novos', 'num', 'numa', 'nunca', 'número', 'nós', 'o', 'obra', 'obrigada', 'obrigado', 'oitava', 'oitavo', 'oito', 'onde', 'ontem', 'onze', 'os', 'ou', 'outra', 'outras', 'outro', 'outros', 'para', 'parece', 'parte', 'partir', 'pela', 'pelas', 'pelo', 'pelos', 'pequena', 'pequenas', 'pequeno', 'pequenos', 'per', 'perante', 'perto', 'pode', 'podem', 'podendo', 'poder', 'poderá', 'poderão', 'poderia', 'poderiam', 'podia', 'podiam', 'pois', 'ponto', 'pontos', 'por', 'porque', 'porquê', 'posição', 'possivelmente', 'possível', 'posso', 'pouca', 'poucas', 'pouco', 'poucos', 'primeira', 'primeiras', 'primeiro', 'primeiros', 'própria', 'próprias', 'próprio', 'próprios', 'próxima', 'próximas', 'próximo', 'próximos', 'pude', 'qual', 'quais', 'qualquer', 'quando', 'quanto', 'quantos', 'quarta', 'quarto', 'quatro', 'que', 'quer', 'querem', 'quero', 'quem', 'questão', 'quinta', 'quinto', 'quinze', 'relação', 'sabe', 'sabem', 'se', 'sebrae', 'segunda', 'segundo', 'sei', 'seis', 'seja', 'sejam', 'sem', 'sempre', 'sendo', 'ser', 'será', 'serão', 'seremos', 'seria', 'seriam', 'seríamos', 'sete', 'seu', 'seus', 'sexta', 'sexto', 'si', 'sido', 'sim', 'sistema', 'só', 'sob', 'sobre', 'somente', 'somos', 'sou', 'sua', 'suas', 'são', 'tal', 'talvez', 'também', 'tampouco', 'tanta', 'tantas', 'tarde', 'tanto', 'te', 'tem', 'tém', 'temos', 'tendo', 'tenha', 'tenham', 'tenhamos', 'tenho', 'ter', 'terá', 'terão', 'terceira', 'terceiro', 'terei', 'teremos', 'teria', 'teriam', 'teríamos', 'teve', 'teu', 'teus', 'tido', 'tinha', 'tinham', 'tínhamos', 'tipo', 'tive', 'tivemos', 'tiver', 'tivera', 'tiveram', 'tivéramos', 'tiverem', 'tivermos', 'tivesse', 'tivessem', 'tivéssemos', 'toda', 'todas', 'todo', 'todos', 'trabalho', 'três', 'treze', 'tu', 'tua', 'tuas', 'tudo', 'um', 'uma', 'umas', 'uns', 'usa', 'usar', 'vai', 'vais', 'vem', 'vendo', 'ver', 'vez', 'vezes', 'vindo', 'vir', 'vários', 'você', 'vocês', 'vos', 'vão', 'www', 'à', 'é', 'última', 'últimas', 'último', 'últimos', 'br', 'ac', 'al', 'ap', 'am', 'ba', 'ce', 'df', 'es', 'go', 'ma', 'mt', 'ms', 'mg', 'pa', 'pb', 'pr', 'pe', 'pi', 'rj', 'rn', 'rs', 'ro', 'rr', 'sc', 'sp', 'se', 'to', 'acre', 'alagoas', 'amazonas', 'bahia', 'ceará', 'distrito', 'espírito', 'santo', 'goiás', 'maranhão', 'mato', 'grosso', 'minas', 'gerais', 'pará', 'paraíba', 'paraná', 'pernambuco', 'piauí', 'rio', 'janeiro', 'grande', 'rondônia', 'roraima', 'santa', 'catarina', 'são', 'paulo', 'sergipe', 'tocantins', 'brasil', 'domingo', 'feira', 'sábado'
]

# Aplicar pré-processamento dos textos: removendo stopwords personalizadas
df['Texto_Processado'] = df['Texto'].apply(lambda x: remover_stopwords(x, stopwords_personalizadas))

# Definir função para realizar a análise de sentimentos
def analisar_sentimento(texto):
    # Ignorar avisos específicos
    warnings.filterwarnings("ignore", category=FutureWarning, module="transformers")
    
    # Tokenizar o texto (Modelo BERT tem um limite de 512 tokens)
    inputs = tokenizer(texto, return_tensors="pt", truncation=True, padding=True, max_length=512)
        
    # Desativar o cálculo de gradiente para economizar memória
    with torch.no_grad():
        # Obter as previsões do modelo
        outputs = model(**inputs)
    
    # Aplicar softmax para obter as probabilidades de cada classe
    probabilidades = torch.nn.functional.softmax(outputs.logits, dim=1)
    
    # Determinar a classe de sentimento com maior probabilidade
    sentimento = torch.argmax(probabilidades, dim=1).item()
    
    return sentimento

# Aplicar a função de análise de sentimentos ao DataFrame
df['Sentimento'] = df['Texto_Processado'].apply(analisar_sentimento)

# Mapear o significado dos sentimentos
df['Significado_Sentimento'] = df['Sentimento'].map(mapa_sentimentos)

# Salvar o DataFrame com os resultados da análise de sentimentos em um novo arquivo Excel
df.to_excel('analise_sentimentos_modelo_BERTimbau_base.xlsx', index=False)

print("Dados salvos no arquivo Excel com sucesso!") 

In [ ]:
# Sentiment Analysis do DataFrame "ws_agencia_sebrae_noticias.xlsx" com o modelo BERTimabu (LARGE)

# Importar as bibliotecas necessárias
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import warnings

# Caminho do arquivo no computador
caminho_arquivo = r'C:\Users\Thairone Almeida\Google Drive\Thairone\Hashtag Treinamentos\Python Impressionador\Script\MBA_USP_ESALQ\Dados até 30 06 2024\ws_agencia_sebrae_noticias.xlsx'

# Caminhos dos modelos BERTimbau
caminho_bertimbau_base = r'C:\Users\Thairone Almeida\Google Drive\Thairone\Hashtag Treinamentos\Python Impressionador\Script\MBA_USP_ESALQ\BERTimbau_base_GoEmotions_portuguese'
caminho_bertimbau_large = r'C:\Users\Thairone Almeida\Google Drive\Thairone\Hashtag Treinamentos\Python Impressionador\Script\MBA_USP_ESALQ\BERTimbau_large_GoEmotions_portuguese'

# Carregar os dados do arquivo Excel
df = pd.read_excel(caminho_arquivo)

# Mapeamento de BERTimbau Base
mapa_sentimentos_base = {
    0: "admiração", 1: "diversão", 2: "raiva", 3: "aborrecimento", 4: "aprovação", 5: "zelo", 6: "confusão", 7: "curiosidade", 8: "desejo", 9: "decepção", 10: "desaprovação", 11: "nojo", 12: "constrangimento", 13: "entusiasmo", 14: "medo", 15: "gratidão", 16: "luto", 17: "alegria", 18: "amor", 19: "nervosismo", 20: "otimismo", 21: "orgulho", 22: "percepção", 23: "alívio", 24: "remorso", 25: "tristeza", 26: "surpresa", 27: "neutro"
}

# Mapeamento de BERTimbau Large
mapa_sentimentos_large = {
    0: "admiração", 1: "diversão", 2: "raiva", 3: "aborrecimento", 4: "aprovação", 5: "zelo", 6: "confusão", 7: "curiosidade", 8: "desejo", 9: "decepção", 10: "desaprovação", 11: "nojo", 12: "constrangimento", 13: "entusiasmo", 14: "medo", 15: "gratidão", 16: "luto", 17: "alegria", 18: "amor", 19: "nervosismo", 20: "otimismo", 21: "orgulho", 22: "realização", 23: "alívio", 24: "remorso", 25: "tristeza", 26: "surpresa", 27: "neutro"
}

# Escolher o modelo que deseja usar (base ou large)
usar_modelo_large = True  # Alterar para True se quiser usar o modelo large

# Carregar o modelo e o tokenizer BERTimbau
if usar_modelo_large:
    tokenizer = BertTokenizer.from_pretrained(caminho_bertimbau_large)
    model = BertForSequenceClassification.from_pretrained(caminho_bertimbau_large)
    mapa_sentimentos = mapa_sentimentos_large
else:
    tokenizer = BertTokenizer.from_pretrained(caminho_bertimbau_base)
    model = BertForSequenceClassification.from_pretrained(caminho_bertimbau_base)
    mapa_sentimentos = mapa_sentimentos_base

# Definir função para remover stopwords personalizadas
def remover_stopwords(texto, stopwords_personalizadas):
    # Garantir que o texto é uma string e tratar valores nulos
    if pd.isnull(texto):
        return ""
    palavras = str(texto).split()
    palavras_filtradas = [palavra for palavra in palavras if palavra.lower() not in stopwords_personalizadas]
    return ' '.join(palavras_filtradas)

# Lista de stopwords personalizadas
stopwords_personalizadas = [
    '0800', 'a', 'as', 'à', 'às', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'acerca', 'adeus', 'agora', 'ainda', 'algo', 'algumas', 'alguns', 'ali', 'além', 'alô', 'ambos', 'ampla', 'amplas', 'amplo', 'amplos', 'ano', 'ante', 'antes', 'ao', 'aos', 'apenas', 'apoio', 'após', 'aqui', 'assim', 'atrás', 'através', 'até', 'aí', 'baixo', 'bem', 'boa', 'boas', 'bom', 'bons', 'breve', 'cada', 'catorze', 'cedo', 'cento', 'certamente', 'certeza', 'cinco', 'cima', 'coisa', 'coisas', 'com', 'como', 'conselho', 'contudo', 'contra', 'cá', 'da', 'das', 'dado', 'dados', 'dela', 'delas', 'dele', 'deles', 'demais', 'debaixo', 'de', 'dentro', 'depois', 'desde', 'dessa', 'dessas', 'desse', 'desses', 'desta', 'destas', 'deste', 'destes', 'devendo', 'deve', 'devem', 'deverá', 'dez', 'dezoito', 'dia', 'diante', 'disse', 'disso', 'disto', 'dito', 'diz', 'dizem', 'dizer', 'do', 'dos', 'dois', 'doze', 'duas', 'dúvida', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'embora', 'em', 'empresa', 'empresas', 'enquanto', 'entre', 'então', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'está', 'estamos', 'estão', 'esta', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'evento', 'exemplo', 'falta', 'favor', 'faz', 'fazem', 'fazemos', 'fazendo', 'fazer', 'fazeres', 'faço', 'feita', 'feitas', 'feito', 'feitos', 'fim', 'final', 'fomos', 'fôramos', 'fora', 'for', 'foram', 'forma', 'formos', 'fosse', 'fossem', 'fôssemos', 'foi', 'fui', 'geral', 'grande', 'grandes', 'grátis', 'haja', 'hajam', 'hajamos', 'hão', 'haver', 'havemos', 'havia', 'hei', 'hoje', 'hora', 'horas', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvéssemos', 'há', 'https', 'isso', 'isto', 'já', 'la', 'lado', 'lhe', 'lhes', 'ligado', 'lo', 'local', 'logo', 'longe', 'lugar', 'lá', 'maior', 'maioria', 'maiorias', 'mal', 'mais', 'mas', 'me', 'meio', 'menor', 'menos', 'meses', 'mesma', 'mesmas', 'mesmo', 'mesmos', 'meu', 'meus', 'minha', 'minhas', 'mil', 'milhões', 'milhares', 'muito', 'muitas', 'muitos', 'na', 'nas', 'nada', 'não', 'naquela', 'naquelas', 'naquele', 'naqueles', 'nas', 'negócio', 'negócios', 'nem', 'nesta', 'nestas', 'nessa', 'nessas', 'nesse', 'nesses', 'ninguém', 'nível', 'no', 'nos', 'noite', 'nome', 'nossa', 'nossas', 'nosso', 'nossos', 'nova', 'novas', 'nove', 'novo', 'novos', 'num', 'numa', 'nunca', 'número', 'nós', 'o', 'obra', 'obrigada', 'obrigado', 'oitava', 'oitavo', 'oito', 'onde', 'ontem', 'onze', 'os', 'ou', 'outra', 'outras', 'outro', 'outros', 'para', 'parece', 'parte', 'partir', 'pela', 'pelas', 'pelo', 'pelos', 'pequena', 'pequenas', 'pequeno', 'pequenos', 'per', 'perante', 'perto', 'pode', 'podem', 'podendo', 'poder', 'poderá', 'poderão', 'poderia', 'poderiam', 'podia', 'podiam', 'pois', 'ponto', 'pontos', 'por', 'porque', 'porquê', 'posição', 'possivelmente', 'possível', 'posso', 'pouca', 'poucas', 'pouco', 'poucos', 'primeira', 'primeiras', 'primeiro', 'primeiros', 'própria', 'próprias', 'próprio', 'próprios', 'próxima', 'próximas', 'próximo', 'próximos', 'pude', 'qual', 'quais', 'qualquer', 'quando', 'quanto', 'quantos', 'quarta', 'quarto', 'quatro', 'que', 'quer', 'querem', 'quero', 'quem', 'questão', 'quinta', 'quinto', 'quinze', 'relação', 'sabe', 'sabem', 'se', 'sebrae', 'segunda', 'segundo', 'sei', 'seis', 'seja', 'sejam', 'sem', 'sempre', 'sendo', 'ser', 'será', 'serão', 'seremos', 'seria', 'seriam', 'seríamos', 'sete', 'seu', 'seus', 'sexta', 'sexto', 'si', 'sido', 'sim', 'sistema', 'só', 'sob', 'sobre', 'somente', 'somos', 'sou', 'sua', 'suas', 'são', 'tal', 'talvez', 'também', 'tampouco', 'tanta', 'tantas', 'tarde', 'tanto', 'te', 'tem', 'tém', 'temos', 'tendo', 'tenha', 'tenham', 'tenhamos', 'tenho', 'ter', 'terá', 'terão', 'terceira', 'terceiro', 'terei', 'teremos', 'teria', 'teriam', 'teríamos', 'teve', 'teu', 'teus', 'tido', 'tinha', 'tinham', 'tínhamos', 'tipo', 'tive', 'tivemos', 'tiver', 'tivera', 'tiveram', 'tivéramos', 'tiverem', 'tivermos', 'tivesse', 'tivessem', 'tivéssemos', 'toda', 'todas', 'todo', 'todos', 'trabalho', 'três', 'treze', 'tu', 'tua', 'tuas', 'tudo', 'um', 'uma', 'umas', 'uns', 'usa', 'usar', 'vai', 'vais', 'vem', 'vendo', 'ver', 'vez', 'vezes', 'vindo', 'vir', 'vários', 'você', 'vocês', 'vos', 'vão', 'www', 'à', 'é', 'última', 'últimas', 'último', 'últimos', 'br', 'ac', 'al', 'ap', 'am', 'ba', 'ce', 'df', 'es', 'go', 'ma', 'mt', 'ms', 'mg', 'pa', 'pb', 'pr', 'pe', 'pi', 'rj', 'rn', 'rs', 'ro', 'rr', 'sc', 'sp', 'se', 'to', 'acre', 'alagoas', 'amazonas', 'bahia', 'ceará', 'distrito', 'espírito', 'santo', 'goiás', 'maranhão', 'mato', 'grosso', 'minas', 'gerais', 'pará', 'paraíba', 'paraná', 'pernambuco', 'piauí', 'rio', 'janeiro', 'grande', 'rondônia', 'roraima', 'santa', 'catarina', 'são', 'paulo', 'sergipe', 'tocantins', 'brasil', 'domingo', 'feira', 'sábado'
]

# Aplicar pré-processamento dos textos: removendo stopwords personalizadas
df['Texto_Processado'] = df['Texto'].apply(lambda x: remover_stopwords(x, stopwords_personalizadas))

# Definir função para realizar a análise de sentimentos
def analisar_sentimento(texto):
    # Ignorar avisos específicos
    warnings.filterwarnings("ignore", category=FutureWarning, module="transformers")
    
    # Tokenizar o texto (Modelo BERT tem um limite de 512 tokens)
    inputs = tokenizer(texto, return_tensors="pt", truncation=True, padding=True, max_length=512)
        
    # Desativar o cálculo de gradiente para economizar memória
    with torch.no_grad():
        # Obter as previsões do modelo
        outputs = model(**inputs)
    
    # Aplicar softmax para obter as probabilidades de cada classe
    probabilidades = torch.nn.functional.softmax(outputs.logits, dim=1)
    
    # Determinar a classe de sentimento com maior probabilidade
    sentimento = torch.argmax(probabilidades, dim=1).item()
    
    return sentimento

# Aplicar a função de análise de sentimentos ao DataFrame
df['Sentimento'] = df['Texto_Processado'].apply(analisar_sentimento)

# Mapear o significado dos sentimentos
df['Significado_Sentimento'] = df['Sentimento'].map(mapa_sentimentos)

# Salvar o DataFrame com os resultados da análise de sentimentos em um novo arquivo Excel
df.to_excel('analise_sentimentos_modelo_BERTimbau_large.xlsx', index=False)

print("Dados salvos no arquivo Excel com sucesso!")